In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import seq2seq
from seq2seq.models import AttentionSeq2Seq, SimpleSeq2Seq, Seq2Seq

from keras.layers import Input, Dense, Lambda, Bidirectional
from keras.layers.embeddings import Embedding
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras import backend as K
from keras import metrics
import tensorflow as tf
from keras.datasets import mnist
from sklearn.model_selection import train_test_split
import pickle

# with open("word_index_conversion.p",'rb') as f:
#     word_index_conversion = pickle.load(f)
    
# word2index = word_index_conversion['word2index']
# index2word = word_index_conversion['index2word']

# lookup_matrix = np.load('word2vec.npy')

lookup_matrix = np.zeros((100,20))

# sentences = np.load('sentences_as_list.npy')

# filtered_sentences = [filter(lambda x:word2index.has_key(x), s) for s in sentences]
# # Filter out sentences with length larger than 2
# filtered_sentences = filter(lambda x:len(x) > 2, filtered_sentences)

# idxes = [[word2index[w] for w in s] for s in filtered_sentences]

# Constants
batch_size = 5
time_steps = 10
embedding_size = lookup_matrix.shape[1]
latent_dim = 2
hidden_dim = 256
epochs = 50
epsilon_std = 1.0
time_steps = 10
epsilon_std = 1

# padded_idxes = pad_sequences(idxes, maxlen=time_steps, dtype='int32', \
#                                  padding='pre', truncating='pre', value=0.)

# x_train, x_test = train_test_split(padded_idxes, test_size=0.2, random_state=42)

x_train, x_test = np.zeros((30,time_steps)), np.zeros((10,time_steps))

optimizer, loss, x, y = SimpleSeq2Seq(output_dim=embedding_size, output_length=time_steps, latent_dim=latent_dim, \
                                      batch_size=batch_size, epsilon_std=epsilon_std, lookup_matrix=lookup_matrix, \
                                      input_shape=(time_steps, embedding_size))

Using TensorFlow backend.


Tensor("TensorArrayReadV3:0", shape=(?, 20), dtype=float32)
Tensor("add_8:0", shape=(5, 2), dtype=float32)
Tensor("Relu:0", shape=(5, 20), dtype=float32)
Tensor("transpose_3:0", shape=(5, ?, 20), dtype=float32)
Tensor("Gather_1:0", shape=(?, 10, 20), dtype=float32)


In [ ]:
with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    epochs = 0
    print_freq = 15
    while epochs < 30:
        print "Epochs:" + str(epochs)
        i = 0
        while i < len(x_train):
            samples = x_train[i:i+batch_size,:]
            i += batch_size
            _, loss_val = sess.run([optimizer, loss], feed_dict={x:samples, y:samples})
            if i % print_freq == 0:
                print loss_val
        epochs += 1

In [ ]:
model = SimpleSeq2Seq(output_dim=embedding_size, output_length=time_steps, input_shape=(time_steps, embedding_size))
model.compile(loss='mse', optimizer='adam')
model.fit(x_train_embedded, x_train_embedded, nb_epoch=100)

In [ ]:
input_length = 10
input_dim = 2

output_length = 8
output_dim = 3

samples = 100

x = np.random.random((samples, input_length, input_dim))
y = np.random.random((samples, output_length, output_dim))

lookup_matrix

models = []
models += [SimpleSeq2Seq(output_dim=output_dim, output_length=output_length, input_shape=(input_length, input_dim))]
models += [SimpleSeq2Seq(output_dim=output_dim, output_length=output_length, input_shape=(input_length, input_dim), depth=2)]

for model in models:
    model.compile(loss='mse', optimizer='sgd')
    model.fit(x, y, nb_epoch=1)
    break

In [ ]:
x.shape

In [ ]:
x.shape

In [ ]:
x.shape

In [ ]:
model.compile(loss='mse', optimizer='sgd')
model.fit(x_train, x_train, nb_epoch=1, validation_data=(x_test, x_test))

In [ ]:
model.input

In [ ]:
input_length = 10
input_dim = 2

output_length = 8
output_dim = 3

samples = 100

model = AttentionSeq2Seq(output_dim=output_dim, output_length=output_length, input_shape=(input_length, input_dim))

In [ ]:
x = Input(batch_shape=(batch_size, time_steps))
x = Embedding(input_dim=lookup_matrix.shape[0], output_dim=lookup_matrix.shape[1], weights=[lookup_matrix])(x)



In [ ]:

h_encoded = LSTM(hidden_dim, batch_input_shape=(batch_size, time_steps, embedding_size), \
                     return_sequences=False, stateful=True)(x)

z_mean = Dense(latent_dim)(h_encoded)
z_log_var = Dense(latent_dim)(h_encoded)

# # build a model to project inputs on the latent space
# encoder = Model(x, z_mean)

# # display a 2D plot of the digit classes in the latent space
# x_test_encoded = encoder.predict(x_test, batch_size=batch_size)
# plt.figure(figsize=(6, 6))
# plt.scatter(x_test_encoded[:, 0], x_test_encoded[:, 1], c=y_test)
# plt.colorbar()
# plt.show()

# # build a digit generator that can sample from the learned distribution
# decoder_input = Input(shape=(latent_dim,))
# _h_decoded = decoder_h(decoder_input)
# _x_decoded_mean = decoder_mean(_h_decoded)
# generator = Model(decoder_input, _x_decoded_mean)

# # display a 2D manifold of the digits
# n = 15  # figure with 15x15 digits
# digit_size = 28
# figure = np.zeros((digit_size * n, digit_size * n))
# # linearly spaced coordinates on the unit square were transformed through the inverse CDF (ppf) of the Gaussian
# # to produce values of the latent variables z, since the prior of the latent space is Gaussian
# grid_x = norm.ppf(np.linspace(0.05, 0.95, n))
# grid_y = norm.ppf(np.linspace(0.05, 0.95, n))

# for i, yi in enumerate(grid_x):
#     for j, xi in enumerate(grid_y):
#         z_sample = np.array([[xi, yi]])
#         x_decoded = generator.predict(z_sample)
#         digit = x_decoded[0].reshape(digit_size, digit_size)
#         figure[i * digit_size: (i + 1) * digit_size,
#                j * digit_size: (j + 1) * digit_size] = digit

# plt.figure(figsize=(10, 10))
# plt.imshow(figure, cmap='Greys_r')
# plt.show()

In [ ]:
def sampling(args):
    z_mean, z_log_var = args         
    epsilon = K.random_normal(shape=(batch_size, latent_dim), mean=0., stddev=epsilon_std)
    return z_mean + K.exp(z_log_var / 2) * epsilon

z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])

h_0 = Dense(hidden_dim, activation='relu')(z)
h_decoded = LSTM(embedding_size, batch_input_shape=(batch_size, time_steps, hidden_dim), \
                     return_sequences=True, stateful=True)(h_0)

x_decoded_mean = Dense(original_dim, activation='sigmoid')(h_decoded)

def vae_loss(x, x_decoded_mean):
    xent_loss = original_dim * metrics.binary_crossentropy(x, x_decoded_mean)
    kl_loss = - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
    return xent_loss + kl_loss

vae = Model(x, x_decoded_mean)
vae.compile(optimizer='adam', loss=vae_loss)

In [ ]:
h_0

In [ ]:
h_encoded

In [ ]:
h_encoded

In [ ]:
vae.fit(x_train, x_train,
        shuffle=True,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(x_test, x_test))

In [ ]:
input_tensor = Input(shape=(36,))
x = Embedding(input_dim=lookup_matrix.shape[0], output_dim=lookup_matrix.shape[1], weights=[lookup_matrix])(input_tensor)

model = Model(input=[input_tensor], output=x)

In [ ]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

In [ ]:
# sess.run(init)

temp = sess.run(h_encoded, feed_dict={model.input:padded_idxes[:10]})

In [ ]:
from keras.layers import Input, Dense, Lambda, LSTM

batch_size = 10
time_steps = 36
embedding_size = 300

h_encoded = LSTM(10, input_shape=(time_steps, embedding_size), \
                     return_sequences=True, stateful=False)(x)